In [1]:
# your_script.py
import os
from helpers import DatasetProcessorData

# Define the directory where the files are saved
load_dir = './data/energy'

# Load each processor's data
mnist_data = DatasetProcessorData.load_data(os.path.join(load_dir, 'mnist_DP.pkl'))
kmnist_data = DatasetProcessorData.load_data(os.path.join(load_dir, 'kmnist_DP.pkl'))
fmnist_data = DatasetProcessorData.load_data(os.path.join(load_dir, 'fmnist_DP.pkl'))
emnist_data = DatasetProcessorData.load_data(os.path.join(load_dir, 'emnist_DP.pkl'))
notmnist_data = DatasetProcessorData.load_data(os.path.join(load_dir, 'notmnist_DP.pkl'))

# Now you can access the results from the data objects
print(f"MNIST max softmax values: {mnist_data.max_softmax_values}")
print(f"KMNIST max softmax values: {kmnist_data.max_softmax_values}")
print(f"FMNIST max softmax values: {fmnist_data.max_softmax_values}")
print(f"EMNIST max softmax values: {emnist_data.max_softmax_values}")
print(f"notMNIST max softmax values: {notmnist_data.max_softmax_values}")

# Example: Print X_test for MNIST
print(f"MNIST X_test: {mnist_data.X_test_jax}")

MNIST max softmax values: [0.99983835 0.9999666  0.9977149  ... 0.9999995  0.9999633  0.96945083]
KMNIST max softmax values: [0.9679183  0.76919377 0.79920596 ... 0.91362894 0.5792618  0.90153044]
FMNIST max softmax values: [0.99945444 0.9753269  0.91486347 ... 0.8308811  0.9994799  0.82589376]
EMNIST max softmax values: [0.5151212  0.8430155  0.7561936  ... 0.97716796 0.9876317  0.9514899 ]
notMNIST max softmax values: [0.67722696 0.9477653  0.52510583 ... 0.7860869  0.9914704  0.8652807 ]
MNIST X_test: [[-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
  -0.42421296]
 [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
  -0.42421296]
 [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
  -0.42421296]
 ...
 [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
  -0.42421296]
 [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
  -0.42421296]
 [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
  -0.42421296]

In [3]:
print(len(mnist_data.max_softmax_values))

8000
